## Fairness Metrics on words

In [9]:
# Importing the required libraries
import numpy as np
import pandas as pd
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
import re
import pickle
import warnings
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_selection import mutual_info_classif

In [10]:
# loading dataset
# importing the dataset
first_data = pd.read_csv('../data/fake_and_real_data.csv')
first_data.head()
second_data = pd.read_csv('../data/WELFake_Dataset.csv')

In [11]:
# Checking for missing values

second_data.isnull().sum()

# Dropping the missing values
second_data = second_data.dropna()
# Checking for duplicates

second_data.duplicated().sum()

# Drop unnamed column
second_data.drop('Unnamed: 0', axis=1, inplace=True)

In [12]:
# concatinating title and text
second_data['text'] = second_data['title'] + ' ' + second_data['text']
# Dropping the title column
second_data = second_data.drop(['title'], axis=1)



In [13]:
first_data.drop_duplicates(inplace=True)


first_data['label_t'] = 0


first_data.head()

,Text,label,label_t
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,0
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,0
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,0


In [14]:
# Preprocessing the data
first_data['label_t'] = (first_data['label'] != 'Fake').astype(int)
first_data = first_data.drop(['label'], axis=1)
first_data = first_data.rename(columns={'label_t': 'label'})
first_data = first_data.rename(columns={'Text': 'text'})
first_data.head()


,text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,0
1,U.S. conservative leader optimistic of common ...,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",1
3,Court Forces Ohio To Allow Millions Of Illega...,0
4,Democrats say Trump agrees to work on immigrat...,1


In [15]:
# Dropping the unnecessary columns
first_data.head()
# converting the text to lowercase
def preprocess(text):
    text = re.sub(r'[^\w\s\']', ' ', text)
    text = re.sub(r' +', ' ', text)
    return text.strip().lower()

first_data['text'] = first_data['text'].apply(preprocess)
second_data['text'] = second_data['text'].apply(preprocess)






In [16]:
first_data.head()
second_data.head()

concat_data = pd.concat([first_data, second_data], axis=0)
concat_data.head()


,text,label
0,top trump surrogate brutally stabs him in the ...,0
1,u s conservative leader optimistic of common g...,1
2,trump proposes u s tax overhaul stirs concerns...,1
3,court forces ohio to allow millions of illegal...,0
4,democrats say trump agrees to work on immigrat...,1


In [17]:
# is trump inclouded 

concat_data['does_contain_trump'] = concat_data['text'].apply(lambda x: 'trump' in x)

In [18]:
concat_data.head()

,text,label,does_contain_trump
0,top trump surrogate brutally stabs him in the ...,0,True
1,u s conservative leader optimistic of common g...,1,True
2,trump proposes u s tax overhaul stirs concerns...,1,True
3,court forces ohio to allow millions of illegal...,0,True
4,democrats say trump agrees to work on immigrat...,1,True


In [19]:
concat_data['does_contain_trump'].value_counts()



does_contain_trump
True     41635
False    39767
Name: count, dtype: int64

In [23]:
def fairness_metrics(df):
    """Calculate fairness for subgroup of population"""
    
    #Confusion Matrix
    cm=confusion_matrix(df['y'],df['y_pred'])
    TN, FP, FN, TP = cm.ravel()
    
    N = TP+FP+FN+TN #Total population
    ACC = (TP+TN)/N #Accuracy
    TPR = TP/(TP+FN) # True positive rate
    FPR = FP/(FP+TN) # False positive rate
    FNR = FN/(TP+FN) # False negative rate
    PPP = (TP + FP)/N # % predicted as positive
    
    return np.array([ACC, TPR, FPR, FNR, PPP])

In [22]:
# trump_unprivileged_group = [{'does_contain_trump': 0}] 
# trump_unprivileged_group

[{'does_contain_trump': 0}]

# Fairness with all values 

In [ ]:
concat_data["y"] = [1 if y == 'Real' else 0 for y in df['y']] 
concat_data["y_pred"] = [model.predict(y, model, tokenizer, device) for y in df['Text']]
result = fairness_metrics(df)
result

# Fairness when having trump 

In [31]:
df_containing_trump = concat_data.loc[concat_data['does_contain_trump'] == True]
result = fairness_metrics(df_containing_trump)
result
# df_containing_trump
# df["does_contain_trump"]
# df

,text,label,does_contain_trump
0,top trump surrogate brutally stabs him in the ...,0,True
1,u s conservative leader optimistic of common g...,1,True
2,trump proposes u s tax overhaul stirs concerns...,1,True
3,court forces ohio to allow millions of illegal...,0,True
4,democrats say trump agrees to work on immigrat...,1,True
...,...,...,...
72124,an unlikely contender rises in france as the a...,0,True
72128,judge jeanine sounds free speech alarm they ar...,1,True
72129,russians steal research on trump in hack of u ...,0,True
72130,watch giuliani demands that democrats apologiz...,1,True


In [32]:
df_without_trump = concat_data.loc[concat_data['does_contain_trump'] == False]
result = fairness_metrics(df_without_trump)
result

,text,label,does_contain_trump
5,france says pressure needed to stop north kore...,1,False
13,watch texas pastor angers parents tells childr...,0,False
24,state department presses north korea to releas...,1,False
35,white house says will work with rubio on child...,1,False
36,republican tax bill retains u s electric vehic...,1,False
...,...,...,...
72125,wow jill stein s fireside chat exposes her del...,1,False
72126,determined to kill can tough gun laws end mass...,0,False
72127,wikileaks email shows clinton foundation funds...,1,False
72131,migrants refuse to leave train at refugee camp...,0,False


In [17]:
# # Simply import it from sklearn
# from sklearn.model_selection import train_test_split

# dataset = concat_data 

# # Separating features from target feature
# features = dataset.columns.tolist()
# features.remove('label')
# target = 'label'
# X = dataset[features]
# y = dataset[target]

# # Define four sets and apply the function
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                     test_size=0.2, # 0.2 indicates a test set size of 20%
#                                                     random_state=42)

In [18]:
# ## applying fairness tests
# from sklearn.feature_extraction.text import TfidfVectorizer 
# # Import the classifier and the metrics from sklearn
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.metrics import accuracy_score, recall_score, f1_score

# dt_clf = DecisionTreeClassifier(random_state=42)
# vect = TfidfVectorizer(stop_words='english', max_df=0.7)
# xv_train = vect.fit_transform(X_train['text'])
# xv_test = vect.transform(X_test['text'])


# print(f"Value to test if testing started")
# # The fit function will do the trick
# dt_clf.fit(xv_train, y_train)

# print(f"Value to test if testing ended")
# # After the training phase, the model will be tested by predicting the values on the test set
# dt_predictions = dt_clf.predict(xv_test)

# dt_accuracy = accuracy_score(y_test, dt_predictions)
# dt_recall = recall_score(y_test, dt_predictions)
# dt_f1_score = f1_score(y_test, dt_predictions)
# print(f"Decision Tree Accuracy: {dt_accuracy}")
# print(f"Decision Tree Recall: {dt_recall}")
# print(f"Decision Tree F1 Score: {dt_f1_score}")

Value to test if testing started
Value to test if testing ended
Decision Tree Accuracy: 0.7108285731834654
Decision Tree Recall: 0.7020305178421242
Decision Tree F1 Score: 0.712821764060022


In [1]:

# from aif360.datasets import BinaryLabelDataset
# from aif360.datasets import StandardDataset
# from aif360.metrics import ClassificationMetric


# # We want to check the fairness level regarding the protected attribute "sex"
# trump_features = ['does_contain_trump']


# dataset = xv_test.toarray()
# dataset['label'] = y_test  # and join the target feature with the others

# predictions = dataset.copy(deep=True)  # we do the same task
# # but this time the target feature is made by the predictions of our model
# predictions['label'] = dt_predictions

# # In this way, we have two datasets. One (dataset) is the original test set containing the original values of features,
# # the other (predictions) contains the original values except for the target one, that is now made of model's predictions

# # These will be used by AIF to compare the classifications of the model with the original values to
# # understand if the model's answers create favouritism toward the privileged attribute


# # This is the object made of the original dataset
# aif_sex_dataset = BinaryLabelDataset(  # Base class for all structured datasets with binary labels.
#     df=dataset,
#     # This means that a prediction is biased toward the privileged attribute if its value is 1 (True)
#     favorable_label=1,
#     unfavorable_label=0,
#     label_names=[target],
#     protected_attribute_names=trump_features,
#     # here we tell AIF that we want to check for predictions
#     privileged_protected_attributes=['does_contain_trump'],
#     # that somehow privilege the attribute "sex_Male"
# )

# # We do the same thing but with the predictions dataset
# aif_sex_pred = BinaryLabelDataset(
#     df=predictions,
#     favorable_label=1,
#     unfavorable_label=0,
#     label_names=[target],
#     protected_attribute_names=trump_features,
#     privileged_protected_attributes=['does_contain_trump'],
# )

# trump_privileged_group = [{'does_contain_trump': 1}]
# trump_unprivileged_group = [{'does_contain_trump': 0}]    

# # We provide the ClassificationMetric object with all the information needed:
# # aif_sex_dataset - The original test set
# # aif_sex_pred - A dataset containing the predictions of the model
# # sex_privileged_group - The privileged group
# # sex_unprivileged_group - The unprivileged group
# fairness_metrics = ClassificationMetric(dataset=aif_sex_dataset,
#                                         classified_dataset=aif_sex_pred,
#                                         unprivileged_groups=trump_unprivileged_group,
#                                         privileged_groups=trump_unprivileged_group)

# # Values less than 0 indicate that privileged group has higher
# # proportion of predicted positive outcomes than unprivileged group.
# # Value higher than 0 indicates that unprivileged group has higher proportion
# # of predicted positive outcomes than privileged group.
# SPD = round(fairness_metrics.statistical_parity_difference(), 3)

# # Measures the deviation from the equality of opportunity, which means that the same
# # proportion of each population receives the favorable outcome. This measure must be equal to 0 to be fair.
# EOD = round(fairness_metrics.equal_opportunity_difference(), 3)

# # Average of difference in False Positive Rate and True Positive Rate for unprivileged and privileged groups
# # A value of 0 indicates equality of odds, which means that samples in both the privileged and unprivileged
# # groups have the same probability of being classified positively.
# AOD = round(fairness_metrics.average_odds_difference(), 3)

# print(f"Statistical Parity Difference (SPD): {SPD}")
# print(f"Equal Opportunity Difference (EOD): {EOD}")
# print(f"Average Odds Difference: {AOD}")

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


NameError: name 'xv_test' is not defined

In [24]:
# import pandas as pd
# import warnings
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE

# warnings.filterwarnings("ignore")
# df = pd.read_csv('WELFake_Dataset.csv')

# # bool_cols = df.select_dtypes(include=['bool']).columns
# # df[bool_cols] = df[bool_cols].astype(int)

# X = df.drop(columns=['label'])
# y = df['label']

# smote = SMOTE(random_state=42)
# X_smote, y_smote = smote.fit_resample(X, y)

# X = X_smote
# y = y_smote

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ValueError: could not convert string to float: 'LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO]'

In [1]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# import xgboost as xgb

# from sklearn.metrics import accuracy_score,confusion_matrix
# from sklearn.metrics import classification_report
# from sklearn.feature_selection import mutual_info_classif

# import sys
# sys.path.insert(0, '..')
# from load_model import load_model, predict
# sys.path

/home/me3eh/Bar_BoRoSa/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['..',
 '/home/me3eh/anaconda3/lib/python311.zip',
 '/home/me3eh/anaconda3/lib/python3.11',
 '/home/me3eh/anaconda3/lib/python3.11/lib-dynload',
 '',
 '/home/me3eh/Bar_BoRoSa/venv/lib/python3.11/site-packages']

In [7]:


# df = pd.read_csv('fake_and_real_data.csv')
# df["y"] = [1 if y == 'Real' else 0 for y in df['label']] 
# df["y_pred"] = ["Real" for y in df['Text']]
# df

,Text,label,y,y_pred
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,Real
1,U.S. conservative leader optimistic of common ...,Real,1,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,Real
4,Democrats say Trump agrees to work on immigrat...,Real,1,Real
...,...,...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,0,Real
9896,Trump consults Republican senators on Fed chie...,Real,1,Real
9897,Trump lawyers say judge lacks jurisdiction for...,Real,1,Real
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,0,Real


In [3]:
# link_to_source = "https://towardsdatascience.com/analysing-fairness-in-machine-learning-with-python-96a9ab0d0705"
# df = pd.read_csv('fake_and_real_data.csv')
# model, tokenizer, device = load_model()

# df_fair = df[['label']]
# df["y"] = [1 if y == 'Real' else 0 for y in df['y']] 
# df["y_pred"] = [model.predict(y, model, tokenizer, device) for y in df['Text']]
# counts = df['text'].value_counts()
# labels = counts.index

# #Plot pie chart
# plt.pie(counts, startangle=90)
# plt.legend(labels, loc=2,fontsize=15)
# plt.title("Truthness of news",size=20)
# result = fairness_metrics(df)

# def fairness_metrics(df):
#     """Calculate fairness for subgroup of population"""
    
#     #Confusion Matrix
#     cm=confusion_matrix(df['y'],df['y_pred'])
#     TN, FP, FN, TP = cm.ravel()
    
#     N = TP+FP+FN+TN #Total population
#     ACC = (TP+TN)/N #Accuracy
#     TPR = TP/(TP+FN) # True positive rate
#     FPR = FP/(FP+TN) # False positive rate
#     FNR = FN/(TP+FN) # False negative rate
#     PPP = (TP + FP)/N # % predicted as positive
    
#     return np.array([ACC, TPR, FPR, FNR, PPP])

SyntaxError: invalid syntax (922732805.py, line 5)